In [ ]:
import os

In [ ]:
# clean last run result

!('./clean.sh')

In [ ]:
# add the package to import path

import sys 
sys.path.append("../../")

In [ ]:
from thunder_ase.fireball import Fireball, MultiFireball
import ase
from ase.io.trajectory import Trajectory
import numpy as np
from ase.io import read
from ase.units import kJ
from ase.eos import EquationOfState

### parameters for Si crystal

In [ ]:
cell = np.array([[2.715000, 2.715000, 0.000000],
                 [2.715000, 0.000000, 2.715000],
                 [0.000000, 2.715000, 2.715000]])
positions = np.array([[0.0000000, 0.0000000, 0.0000000],
                      [1.3575000, 1.3575000, 1.3575000]])

### parameters for fireball

In [ ]:
# set Fdata dir
Fdata_path='~/Fdata/Fdata-McWEDA-0.15-3SN.Sis4.8p5.35/'

In [ ]:
kwargs = {'kpt_size': [3, 3, 3],
          'efermi_T': 200.0,
          }

In [ ]:
# scale the cell from 0.8 to 1.2
cell_factors = np.linspace(0.8, 1.2, 8)

## Run Fireball

There are two way to run fireball calculator

* serial mode by Fireball calculator
* multi-atoms mode by MultiFireball calculator

We recommend multi-atoms mode, which saves Fdata reading time.

### 1. Run fireball in series mode

In [ ]:
# save the trajectory during calculation

traj = Trajectory('Si.traj', 'w')

# main loop, will takes several minutes

for cf in cell_factors:
    atoms = ase.Atoms(numbers=[14, 14],
                      cell=cell,
                      pbc=True,
                      positions=positions,
                      )
    atoms.set_cell(cell=cell*cf, scale_atoms=True)
    calc = Fireball(command='~/bin/fireball-ase.3.x', 
                    Fdata_path=Fdata_path,
                    **kwargs)
    atoms.set_calculator(calc)
    e0 = atoms.get_potential_energy()
    traj.write(atoms)
    print("The energy for cell factor {:.3f} is {:.3f}".format(cf, e0))

In [ ]:
# read the atoms
configs = read('Si.traj', index=':')
volumes = [si.get_volume() for si in configs]
energies = [si.get_potential_energy() for si in configs]
# fit Equation of State
eos = EquationOfState(volumes, energies)
v0, e0, B = eos.fit()
# plot result
eos.plot()

### 2. Run fireball in multi-atoms mode

In [ ]:
atoms_list = []

for cf in cell_factors:
    atoms = ase.Atoms(numbers=[14, 14],
                      cell=cell,
                      pbc=True,
                      positions=positions,
                      )
    atoms.set_cell(cell=cell*cf, scale_atoms=True)
    calc = Fireball(command='~/bin/fireball-ase.3.x', 
                    Fdata_path=Fdata_path,
                    **kwargs)
    atoms.set_calculator(calc)
    atoms_list.append(atoms)

# set up multi-fireball calculator

multi_calc = MultiFireball(atoms_list=atoms_list)
multi_calc.write_input()
multi_calc.calculate()

# save the trajectory during calculation

traj = Trajectory('Si_multicalc.traj', 'w')
_ = [traj.write(atoms)  for atoms in atoms_list]

# print e0
e0_list = [atoms.get_potential_energy() for atoms in atoms_list]
for cf, e0 in zip(cell_factors, e0_list):
    print("The energy for cell factor {:.3f} is {:.3f}".format(cf, e0)) 

In [ ]:
# read the atoms
configs = read('Si_multicalc.traj', index=':')
volumes = [si.get_volume() for si in configs]
energies = [si.get_potential_energy() for si in configs]
# fit Equation of State
eos = EquationOfState(volumes, energies)
v0, e0, B = eos.fit()
# plot result
eos.plot()